In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import dask.dataframe as ddf
import matplotlib.pyplot as plt
import seaborn as sns

# plt.style.use("classic")
# %matplotlib inline
sns.set_style('darkgrid')

In [ ]:
db = sqlite3.connect('../input/california-traffic-collision-data-from-switrs/switrs.sqlite')
cur = db.cursor()
tables = [name[0] for name in cur.execute("SELECT name FROM sqlite_master")]
tables = ['collisions', 'parties']

# 1. EDA

In [ ]:
cols = {
    "collisions":
    [
        "case_id"
        
        # Location
        ,"county_location"
        ,"population"
        
        # Day of Week
        # At Night/Time of Day
        # COVID-19
        ,"collision_date"
        
        # Weather
        ,"injured_victims"
        ,"weather_1"
        ,"weather_2"
        ,"lighting"
        
        # Drugs or Alcohol
        ,"alcohol_involved"
    ],
    
    "parties":
    [
        "case_id"
        ,"party_number"
        
        # Who is usually at fault
        ,"at_fault"
        
        # Demographics
        ,"party_age"
        ,"party_sex"
        
        # Drugs or Alcohol
        ,"party_drug_physical"
    ],
}

def get_str_list(list):
    string = ""
    for col in list:
        string += col + ", "
    string = string[:-2]
    return string

df = {}

In [ ]:
# Read collisions table
cmd = "SELECT " + get_str_list(cols["collisions"]) + " FROM collisions"
df["collisions"] = pd.DataFrame([list(x) for x in cur.execute(cmd)], columns=cols["collisions"])

In [ ]:
display(df["collisions"].head())
df["collisions"].info()

In [ ]:
# Read parties table
cmd = "SELECT " + get_str_list(cols["parties"]) + " FROM parties"
df["parties"] = pd.DataFrame([list(x) for x in cur.execute(cmd)], columns=cols["parties"])

In [ ]:
display(df["parties"].head())
df["parties"].info()

# 2. Feature Engineering

In [ ]:
# Feature engineering collisions df
df["collisions"]["collision_date"] = pd.to_datetime(df["collisions"]["collision_date"])
df["collisions"]["collision_year_month"] = df["collisions"]["collision_date"].dt.to_period('M')
df["collisions"]["collision_year"] = df["collisions"]["collision_date"].dt.year
df["collisions"]["collision_month"] = df["collisions"]["collision_date"].dt.month
df["collisions"]["collision_hour"] = df["collisions"]["collision_date"].dt.hour
df["collisions"]["collision_dow"] = df["collisions"]["collision_date"].dt.day_name()
# df["collisions"] = df["collisions"][df["collisions"]["collision_year"] == 2020]
df["collisions"].head()

# 3. Answering Questions

### 1. When do collisions usually happen? Are they linked to a day of the week when users are more likely to be driving under the influence of drugs or alcohol?

In [ ]:
df_dow_cases = df["collisions"].groupby('collision_dow')['case_id'].nunique().reset_index()
df_dow_cases_alcohol = df["collisions"][df["collisions"]["alcohol_involved"]==1].groupby('collision_dow')['case_id'].nunique().reset_index()
df_dow_cases = df_dow_cases.rename(columns={'collision_dow': 'Day of Week', 'case_id': 'Cases'})
df_dow_cases_alcohol = df_dow_cases_alcohol.rename(columns={'collision_dow': 'Day of Week', 'case_id': 'Cases with Alcohol'})

# Visualize
sns.barplot(x="Day of Week", y="Cases", data=df_dow_cases, palette="hls")
plt.show()
sns.barplot(x="Day of Week", y="Cases with Alcohol", data=df_dow_cases_alcohol, palette="hls")
plt.show()

### 2. Who is usually at fault?  Are the specific demographics(age/gender) that are more likely to be involved in accidents?

In [ ]:
display(df["parties"].groupby('at_fault')['case_id'].nunique())
display(df["parties"].groupby('at_fault')['party_age'].mean())

df_q2_1 = df["parties"][df["parties"]['party_sex']!='X'].groupby(['at_fault', 'party_sex'])['case_id'].nunique().reset_index()
df_q2_2 = df["parties"][df["parties"]['party_sex']!='X'].groupby(['at_fault', 'party_sex'])['party_age'].mean().reset_index()
df_q2_1 = df_q2_1.rename(columns={'at_fault': 'At Fault', 'party_sex': 'Sex', 'case_id': 'Count'})
df_q2_2 = df_q2_2.rename(columns={'at_fault': 'At Fault', 'party_sex': 'Sex', 'party_age': 'Party Age'})

# display(df_q2_1)
# display(df_q2_2)

# Visualize
sns.barplot(x="At Fault", y="Count", hue="Sex", data=df_q2_1, palette="hls")
plt.show()
sns.barplot(x="At Fault", y="Party Age", hue="Sex", data=df_q2_2, palette="hls")
plt.show()

del df_q2_1
del df_q2_2

### 3. Does the weather have an effect on the collisions? Are there seasons where collisions are more likely to occur?

In [ ]:
# display(df["collisions"].groupby(['weather_1','weather_2','lighting'])['case_id'].nunique())
display(df["collisions"].groupby('lighting')['case_id'].nunique().reset_index().sort_values(by=['case_id'], ascending=False))
display(df["collisions"].groupby('weather_1')['case_id'].nunique().reset_index().sort_values(by=['case_id'], ascending=False))
display(df["collisions"].groupby('weather_2')['case_id'].nunique().reset_index().sort_values(by=['case_id'], ascending=False))

### 4. Which locations have a lot of collisions? Are there locations that become more accident prone at night?

In [ ]:
df["collisions"][df["collisions"]["lighting"]!="daylight"].groupby('county_location')['injured_victims'].sum().reset_index().sort_values(by=['injured_victims'], ascending=False)
# share as % of population

### 5. What is the effect of COVID-19 on the collisions? Are roads safer due to COVID?

In [ ]:
df_cases_overtime = df["collisions"][df["collisions"]["collision_year"]>=2019].groupby("collision_year_month")["case_id"].nunique().reset_index()
df_cases_overtime = df_cases_overtime.rename(columns={'collision_year_month': 'Year-Month', 'case_id': 'Case Count'})

plt.figure(figsize = (15,8))
ax = sns.barplot(x='Year-Month', y='Case Count',data=df_cases_overtime)
ax.set_xticklabels(ax.get_xticklabels(), rotation = 30)

del df_cases_overtime

### EXTRA QUESTIONS: Which vehicle type causes the most accidents?